In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_dir = '/content/drive/My Drive/weather classification/train'
val_dir = '/content/drive/My Drive/weather classification/validation'
test_dir = '/content/drive/My Drive/weather classification/test'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Task 1
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Load data
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'
)

# Flow validation images in batches
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'
)

# Flow test images in batches
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'
)

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

Found 400 images belonging to 4 classes.
Found 200 images belonging to 4 classes.
Found 525 images belonging to 4 classes.
Epoch 1/10
13/13 [==============================] - 131s 10s/step - loss: 1.1925 - accuracy: 0.4150 - val_loss: 0.7984 - val_accuracy: 0.7200
Epoch 2/10
13/13 [==============================] - 20s 1s/step - loss: 0.6580 - accuracy: 0.7450 - val_loss: 0.8463 - val_accuracy: 0.6950
Epoch 3/10
13/13 [==============================] - 20s 1s/step - loss: 0.6035 - accuracy: 0.7875 - val_loss: 0.5248 - val_accuracy: 0.7800
Epoch 4/10
13/13 [==============================] - 23s 2s/step - loss: 0.3755 - accuracy: 0.8750 - val_loss: 0.6849 - val_accuracy: 0.7850
Epoch 5/10
13/13 [==============================] - 22s 2s/step - loss: 0.3194 - accuracy: 0.8875 - val_loss: 0.7741 - val_accuracy: 0.7500
Epoch 6/10
13/13 [==============================] - 23s 2s/step - loss: 0.2323 - accuracy: 0.9300 - val_loss: 0.4741 - val_accuracy: 0.8150
Epoch 7/10
13/13 [=================

NameError: name 'test_data' is not defined

In [ ]:
# Evaluate the model on test set
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

 5/17 [=======>......................] - ETA: 1:41 - loss: 0.5356 - accuracy: 0.8438

In [ ]:
# Task 2

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# No Augmentation for validation and test data
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'
)

# Flow validation images in batches
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'
)

# Flow test images in batches
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'
)

# Train the model with augmented data
history_augmented = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // 32,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.n // 32
)

# Evaluate the model on test set
test_loss_augmented, test_acc_augmented = model.evaluate(test_generator)
print('Test accuracy with augmentation:', test_acc_augmented)

In [ ]:
# Task 3

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_height, image_width, 224))

# Freeze the convolutional base
base_model.trainable = False

# Add custom classifier on top
x = Flatten()(base_model.output)
x = Dense(224, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(224, activation='softmax')(x)

# Compile the model
model = Model(base_model.input, output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_pretrained = model.fit(train_generator, epochs=10, validation_data=val_generator)

# Evaluate the model on test set
test_loss_pretrained, test_acc_pretrained = model.evaluate(test_generator)
print('Test accuracy with pre-trained network:', test_acc_pretrained)